In [1]:
# ===================== INSTALL DEPENDENCIES =====================
!pip install -q langchain sentence-transformers faiss-cpu pypdf langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [2]:
# ===================== IMPORTS =====================
import os
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from sentence_transformers.cross_encoder import CrossEncoder
from IPython.display import display, Markdown
import numpy as np

In [3]:
# ===================== LOAD PDF =====================
loader = PyPDFLoader("/content/solid-python.pdf")
documents = loader.load()

# Assign parent IDs
for i, doc in enumerate(documents):
    doc.metadata["doc_id"] = f"doc_{i}"

In [4]:
# ===================== SPLITTING =====================
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter

In [5]:
# ===================== EMBEDDINGS =====================
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_model

/tmp/ipython-input-5-2072525915.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
# ===================== VECTOR STORE + DOC STORE =====================
# Create a FAISS vector store from the documents and embeddings
vectorstore = FAISS.from_documents(documents, embedding_model)
docstore = InMemoryStore()  # Stores parent docs

Basic kNN/Similarity-based retrieval from embeddings, preffered on Small to medium documents, can be used on Any Vector DB

In [7]:
# =====================  RETRIEVER WITH CUSTOM MMR =====================
retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"k": 15, "fetch_k": 30}, lambda_mult=0.3
)

###MMR (Maximal Marginal Relevance) is a ranking algorithm used in retrieval systems to balance:
1. Relevance (how similar the document is to the query)
2. Diversity (how different it is from already selected documents)
####lambda_mult
** λ Value	Behavior <br/>
** λ = 1.0	Only relevance (normal similarity search)<br/>
** λ = 0.0	Only diversity (may miss relevance)<br/>
** λ = 0.3–0.7	Balanced mix (recommended range)<br/>

In [8]:
# ===================== ADD DOCUMENTS =====================
retriever.add_documents(documents)

['99542406-d16c-4f7f-bda6-76ebd67e9e5d',
 'db66832f-0c36-4e87-a785-0d5b042e5976',
 'f5efb6b6-ab8c-4741-8257-62249176f669',
 '6e048835-3a99-4e66-a321-8ed213760430',
 '2320e6a4-928d-4113-b233-766fe4db1b18',
 'ecde01b7-f706-4883-9e9d-e11658ac8531',
 '679d4a86-964b-4888-ac3b-4daa7f954e8e',
 'c53783c5-aaca-4d10-9e17-ec23b3868b6b',
 '200baa6c-b14a-4cd5-9b8a-fb137d783a0d',
 'cf7b36d5-49dd-418a-b0e8-e09340c47a8a',
 'f9119988-80d8-4680-9799-5c1503b2d975',
 '473bc23c-ef50-41f0-b247-d1561a1e36a4',
 '19443b32-7bfe-4d9c-9188-b140abe64a71',
 '94c5c05d-b8ec-4556-9f07-0f5f6570fb95',
 '98089423-9e73-48af-8abf-c9153c75e9e4',
 '181819ab-e5f1-4776-9b5b-ae198f918809',
 'd79fe23b-6c1b-43bf-9c96-13b946a86656',
 'f38b1533-bad0-4f2b-8097-dbc46a7a92e3',
 '622cd0a4-3584-4081-ba88-cf0e38985475',
 'fde67642-1257-4bff-b481-42fc05a3a63b',
 '3acf42a0-c56f-4da1-8dc7-970652cde4a6',
 '330b156f-029b-4773-98d4-51673b60e1b8']

In [9]:
# ===================== LLM SETUP =====================
from google.colab import userdata
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=userdata.get("GROQ_API_KEY")
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bc57e7b82d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bc57e33bf50>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
# ===================== PROMPT =====================
prompt_template = PromptTemplate.from_template(
    "Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
)

In [11]:
# ===================== BUILD RAG CHAIN =====================
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bc57e7b82d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bc57e33bf50>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7bc5874b0250>, search_type='mmr', search_kwarg

In [12]:
# ===================== RUN A QUERY =====================
question = "What is the main objective of the document?"
result = qa_chain.invoke({"query": question})
print(" Answer:", result["result"])

 Answer: The main objective of the document is to discuss and explain the SOLID principles of software design, which aim to maintain software quality over time by keeping software applications flexible, robust, reusable, and developable. The document provides an overview of the five principles: Single Responsibility, Open-Closed, Liskov Substitution, Interface Segregation, and Dependency Inversion, and discusses how they can help prevent "software rot" and promote good software design.


In [13]:
# ===================== METRIC EVALUATION HELPERS =====================
def precision_recall_f1(retrieved_ids, relevant_ids):
    intersection = set(retrieved_ids).intersection(set(relevant_ids))
    precision = len(intersection) / len(retrieved_ids) if retrieved_ids else 0
    recall = len(intersection) / len(relevant_ids) if relevant_ids else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision + recall) > 0 else 0
    return precision, recall, f1

def mean_reciprocal_rank(retrieved_ids, relevant_ids):
    for i, doc_id in enumerate(retrieved_ids):
        if doc_id in relevant_ids:
            return 1 / (i + 1)
    return 0.0

def hit_at_k(retrieved_ids, relevant_ids, k):
    return int(any(doc_id in retrieved_ids[:k] for doc_id in relevant_ids))

def faithfulness_check(llm, answer, context_docs):
    context = "\n\n".join([doc.page_content for doc in context_docs[:5]])
    prompt = f"Context:\n{context}\n\nAnswer:\n{answer}\n\nIs the answer supported by the context? Answer 'yes' or 'no'."
    verdict = llm.invoke(prompt)
    return 1 if "yes" in verdict.content.lower() else 0

In [14]:
# ===================== QUERY SETUP =====================
queries = [
    "What are the five solid principles?",
    "Why is dependency inversion important?",
    "What is the difference between interface segregation and Liskov substitution?",
]

ground_truths = [
    ["doc_3", "doc_7"],     # Expected relevant docs for query 1
    ["doc_5"],              # Expected relevant docs for query 2
    ["doc_6", "doc_8"],     # Expected relevant docs for query 3
]

In [15]:
# ===================== RUN EVALUATION =====================
all_precisions, all_recalls, all_f1s, all_mrrs, all_hits, all_faith = [], [], [], [], [], []

for i, question in enumerate(queries):
    print(f"\n Query: {question}")

    retrieved_docs = retriever.get_relevant_documents(question)
    retrieved_ids = [doc.metadata.get("doc_id") for doc in retrieved_docs]

    precision, recall, f1 = precision_recall_f1(retrieved_ids, ground_truths[i])
    mrr = mean_reciprocal_rank(retrieved_ids, ground_truths[i])
    hit = hit_at_k(retrieved_ids, ground_truths[i], k=5)

    answer = qa_chain.invoke({"query": question})
    is_faithful = faithfulness_check(llm, answer["result"], retrieved_docs)

    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)
    all_mrrs.append(mrr)
    all_hits.append(hit)
    all_faith.append(is_faithful)

    print(f"Answer: {answer['result']}")
    print(f"Precision: {precision:.2f} | Recall: {recall:.2f} | F1: {f1:.2f} | MRR: {mrr:.2f} | Hit@5: {hit} | Faithful: {is_faithful}")


 Query: What are the five solid principles?


/tmp/ipython-input-15-2101565423.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


Answer: The five SOLID principles are:

1. **S** - Single Responsibility Principle
2. **O** - Open-Closed Principle
3. **L** - Liskov Substitution Principle
4. **I** - Interface Segregation Principle
5. **D** - Dependency Inversion Principle
Precision: 0.07 | Recall: 0.50 | F1: 0.12 | MRR: 0.08 | Hit@5: 0 | Faithful: 1

 Query: Why is dependency inversion important?
Answer: Dependency inversion is important because it helps to achieve the aims of keeping software applications flexible, robust, reusable, and developable. By inverting dependencies, high-level modules are no longer dependent on low-level modules, but rather both depend on abstractions. This makes the software less rigid, less fragile, and less prone to immobility and viscosity, which are symptoms of rotting software design.

In the context of the provided text, dependency inversion is demonstrated through an example where a `get_names` function is dependent on a specific database connection. By introducing an abstract `Da

In [16]:
# ===================== METRIC SUMMARY =====================
print("\n Average Metrics Across Queries")
print(f"Precision: {np.mean(all_precisions):.2f}")
print(f"Recall: {np.mean(all_recalls):.2f}")
print(f"F1 Score: {np.mean(all_f1s):.2f}")
print(f"MRR: {np.mean(all_mrrs):.2f}")
print(f"Hit@5: {np.mean(all_hits):.2f}")
print(f"Faithfulness: {np.mean(all_faith):.2f}")


 Average Metrics Across Queries
Precision: 0.04
Recall: 0.50
F1 Score: 0.08
MRR: 0.36
Hit@5: 0.33
Faithfulness: 1.00
